## T-test : Effect of habitat type on life strategy

### Objective:
Determine if habitat type (aquatic and terrestrial) has a significant effect on life strategy (PC1)

In [ ]:
import pandas as pd
import plotly.express as px
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as mc
from statsmodels.stats.multicomp import tukeyhsd
from scipy.stats import ttest_ind

In [71]:
df= pd.read_csv('../data/new_habitat.csv')
display(df.head())

aquatic = df[df['habitat'] == "A"]['PC1']
terrestrial = df[df['habitat'] == "T"]['PC1']

,Species,PC1,habitat
0,Symsagittifera_roscoffensis,-4.332599,A
1,Aequipecten_opercularis,-2.418778,A
2,Mimachlamys_varia,-2.773367,A
3,Mytilus_edulis,-2.602300,A
4,Panopea_abbreviata,-0.460463,A


Hypothesis
- H0: No significant effect of the habitat
- H1: There is a significant effect of the habitat

In [72]:
# Test for normality
stat, p = stats.shapiro(df['PC1'])  
print(f"Shapiro-Wilk test: Statistic={stat}, p-value={p}")

Shapiro-Wilk test: Statistic=0.9579976327808482, p-value=2.628020458087804e-17


p-value = 2.63e-17 < 0.05 -> the data does not follow a normal distribution

We still do the t-test

In [73]:
fig = px.box(
    df,
    x="habitat",         
    y="PC1",           
    color="habitat",
    color_discrete_sequence=px.colors.qualitative.Set2,
    # category_orders={"Class": ordered_classes},
)

fig.update_layout(
    title="Boxplot of PC1 by habitat",
    xaxis_title="Habitat",
    yaxis_title="PC1",
    height=600
)

fig.show()

In [74]:
# t-test
stat, p = stats.ttest_ind(aquatic,
                          terrestrial,
                          equal_var=True)  
print(f"Independent t-test: Statistic={stat}, p-value={p}")

Independent t-test: Statistic=-5.035889812391533, p-value=5.56338439116844e-07


p-value = 5.56e-07 < 0.05 -> There is a significant difference in PC1 between aquatic and terrestrial habitat

The negative sign suggests PC1 is lower for aquatic habitats compared to terrestrial ones and that corresponds to the plot



## ANOVA: Effect of class on PC1

In [75]:
df_species = pd.read_csv("../data/species.csv")
df_pca = pd.read_csv("../data/pca.csv")

df = pd.merge(df_species, df_pca, on='Species')
df = df.drop(columns=["ScientificName", "Mod", "MRE", "SMSE", "COM", "rest", "PC2"])
chordata_data = df[df['Phylum'] == 'Chordata']

chordata_data.head()

,Species,Phylum,Class,Order,Family,CommonName,PC1
66,Branchiostoma_floridae,Chordata,Leptocardii,Amphioxiformes,Branchiostomidae,Florida lancelet,-2.659253
67,Ascidiella_aspersa,Chordata,Ascidiacea,Phlebobranchia,Ascidiidae,European sea squirt,-3.010048
68,Boltenia_echinata,Chordata,Ascidiacea,Stolidobranchia,Pyuridae,Cactus sea squirt,-1.231608
69,Styela_plicata,Chordata,Ascidiacea,Stolidobranchia,Styelidae,Pleated sea squirt,-3.595991
70,Hemiscyllium_ocellatum,Chordata,Chondrichthyes,Orectolobiformes,Hemiscylliidae,Epaulette shark,2.833690


In [76]:
# boxplot
ordered_classes = (
    chordata_data.groupby("Class")["PC1"]
    .median()
    .sort_values()
    .index.tolist()
)

fig = px.box(
    chordata_data,
    x="Class",         
    y="PC1",           
    color="Class",     
    color_discrete_sequence=px.colors.qualitative.Set2,
    # category_orders={"Class": ordered_classes},
)

fig.update_layout(
    title="Boxplot of PC1 by class",
    xaxis_title="Class",
    yaxis_title="PC1",
    height=600
)

fig.show()
# fig.write_html("../figures/boxplot_class.html")

In [77]:
# Anova
model = smf.ols('PC1 ~ Class', data=chordata_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

               sum_sq      df           F         PR(>F)
Class     1273.153491     7.0  152.330453  2.618877e-154
Residual  1234.572306  1034.0         NaN            NaN


p-value < 0,05 - > There is a significant effect of taxonomic classes on life strategy 

This suggests that at least one taxonomic class show a statistically different life strategy compared to others within Chordata

In [78]:
# Tukey's test

tukey = mc.MultiComparison(chordata_data["PC1"], chordata_data["Class"])
tukey_result = tukey.tukeyhsd()

print(tukey_result)

        Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1         group2     meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------
Actinopterygii       Amphibia   0.0275    1.0 -0.3009  0.3559  False
Actinopterygii     Ascidiacea  -2.2444 0.0095 -4.1645 -0.3244   True
Actinopterygii           Aves   2.4183    0.0  1.8855  2.9511   True
Actinopterygii Chondrichthyes   4.2337    0.0  3.6249  4.8424   True
Actinopterygii    Leptocardii  -2.2911 0.4186 -5.6121  1.0298  False
Actinopterygii       Mammalia   2.3811 0.0444  0.0312   4.731   True
Actinopterygii       Reptilia   2.3892    0.0  2.0677  2.7108   True
      Amphibia     Ascidiacea   -2.272 0.0093  -4.212 -0.3319   True
      Amphibia           Aves   2.3908    0.0  1.7898  2.9918   True
      Amphibia Chondrichthyes   4.2062    0.0   3.537  4.8754   True
      Amphibia    Leptocardii  -2.3187  0.407 -5.6512  1.0139  False
      Amphibia       Mammalia   2.

In [79]:
group_letters = {
    'Actinopterygii': 'a',
    'Amphibia': 'ab',
    'Leptocardii': 'ac',
    'Ascidiacea': 'c',
    'Aves': 'd',
    'Mammalia': 'bde',
    'Chondrichthyes': 'e',
    'Reptilia': 'd'
}

In [80]:
fig = px.box(
    chordata_data,
    x="Class",         
    y="PC1",           
    color="Class",     
    color_discrete_sequence=px.colors.qualitative.Set2,
    # category_orders={"Class": ordered_classes},
)

y_positions = chordata_data.groupby("Class")["PC1"].max() + 0.5  

for cls, y in y_positions.items():
    fig.add_annotation(
        x=cls,
        y=y,
        text=group_letters.get(cls, ''),
        showarrow=False,
        font=dict(size=14, color='black'),
        yanchor="bottom"
    )

fig.update_layout(
    title="Boxplot of PC1 by class",
    xaxis_title="Class",
    yaxis_title="PC1",
    height=600
)

fig.show()
#save
fig.write_html("../figures/boxplot_class_tukey.html")


In [81]:
import scikit_posthocs as sp

sp.posthoc_tukey(chordata_data, val_col="PC1", group_col="Class")


,Leptocardii,Ascidiacea,Chondrichthyes,Actinopterygii,Amphibia,Reptilia,Aves,Mammalia
Leptocardii,1.000000e+00,1.000000e+00,1.567790e-07,0.418560,0.407008,5.718721e-04,5.918434e-04,0.011733
Ascidiacea,1.000000e+00,1.000000e+00,0.000000e+00,0.009541,0.009326,2.131451e-11,5.083356e-11,0.000108
Chondrichthyes,1.567790e-07,0.000000e+00,1.000000e+00,0.000000,0.000000,0.000000e+00,1.469133e-10,0.281432
Actinopterygii,4.185598e-01,9.541362e-03,0.000000e+00,1.000000,0.999997,0.000000e+00,0.000000e+00,0.044420
Amphibia,4.070085e-01,9.326181e-03,0.000000e+00,0.999997,1.000000,0.000000e+00,0.000000e+00,0.052418
Reptilia,5.718721e-04,2.131451e-11,0.000000e+00,0.000000,0.000000,1.000000e+00,9.999999e-01,1.000000
Aves,5.918434e-04,5.083356e-11,1.469133e-10,0.000000,0.000000,9.999999e-01,1.000000e+00,1.000000
Mammalia,1.173325e-02,1.078110e-04,2.814323e-01,0.044420,0.052418,1.000000e+00,1.000000e+00,1.000000
